## Librairies

In [1]:
import os 
from PIL import Image
import cv2
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib
import matplotlib.pyplot as plt
import imageio
import csv

from skimage.measure import ransac
from skimage.transform import ProjectiveTransform
from skimage.transform import AffineTransform

from pathlib import Path
import random
import matplotlib.cm as cm
import torch

from models.matching import Matching

torch.set_grad_enabled(False)


## Parametres

In [2]:
# Parametres

cache=False
eval=False
fast_viz=False
force_cpu=False
input_dir='assets/pair/'
input_pairs='assets/pair.txt'
keypoint_threshold=0.005
match_threshold=0.2
max_keypoints=1024
max_length=-1
nms_radius=4
opencv_display=False
output_dir='resultat'
resize=[640, 480]
resize_float=False
show_keypoints=False
shuffle=False
sinkhorn_iterations=20
superglue='outdoor'
viz=True
viz_extension='jpg'

if (len(resize) == 2) and (resize[1] == -1):    
    resize = resize[0:1]
if len(resize) == 2:
    print('Will resize to {}x{} (WxH)'.format(
        resize[0], resize[1]))
elif len(resize) == 1 and resize[0] > 0:
    print('Will resize max dimension to {}'.format(resize[0]))
elif len(resize) == 1:
    print('Will not resize images')
else:
    raise ValueError('Cannot specify more than two integers for --resize')
    
# Load the SuperPoint and SuperGlue models.
device = 'cuda' if torch.cuda.is_available() and not force_cpu else 'cpu'
print('Running inference on device \"{}\"'.format(device))
config = {
    'superpoint': {
        'nms_radius': nms_radius,
        'keypoint_threshold': keypoint_threshold,
        'max_keypoints': max_keypoints
    },
    'superglue': {
        'weights': superglue,
        'sinkhorn_iterations': sinkhorn_iterations,
        'match_threshold': match_threshold,
    }
}

matching = Matching(config).eval().to(device)

Will resize to 640x480 (WxH)
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


## Fonctions

In [3]:
def filteringBlackAreaKeypoints(img,keypoints,keypoints1):
    new_keypoints = [[1,1]]
    new_keypoints1 = [[1,1]]
    
#     print('img.shape', img.shape)
    
    #print('keypoints : ',keypoints)

    for i in range(len(keypoints)):
        kpt_x = int(keypoints[i][0])
        kpt_y = int(keypoints[i][1])
        img_kpt = img[(kpt_y-2):(kpt_y+3),(kpt_x-2):(kpt_x+3)]
        
        nbr_zeros=0
        
        for m in range(img_kpt.shape[0]):
            for n in range(img_kpt.shape[1]):
                if(img_kpt[m,n]==0):
                    nbr_zeros += 1
                                
#         sum_masque = np.sum(img_kpt)
#         print('sum : ',sum_masque)
#         if(sum_masque >  2000):
        #print('nbr_zeros :',nbr_zeros)
        if (nbr_zeros<1):
#             plt.figure(figsize=(10, 10))
#             plt.imshow(img_kpt)
#             plt.axis('off')
#             plt.show()
            #print(img_kpt)
            new_keypoints = np.concatenate((new_keypoints,[keypoints[i]]),axis=0)
            new_keypoints1 = np.concatenate((new_keypoints1,[keypoints1[i]]),axis=0)
        
    #print('new_keypoints[1:] : ',new_keypoints[1:])
            
    return new_keypoints[1:],new_keypoints1[1:]

def process_resize(w, h, resize):
    assert(len(resize) > 0 and len(resize) <= 2)
    if len(resize) == 1 and resize[0] > -1:
        scale = resize[0] / max(h, w)
        w_new, h_new = int(round(w*scale)), int(round(h*scale))
    elif len(resize) == 1 and resize[0] == -1:
        w_new, h_new = w, h
    else:  # len(resize) == 2:
        w_new, h_new = resize[0], resize[1]

    # Issue warning if resolution is too small or too large.
    if max(w_new, h_new) < 160:
        print('Warning: input resolution is very small, results may vary')
    elif max(w_new, h_new) > 2000:
        print('Warning: input resolution is very large, results may vary')

    return w_new, h_new

def frame2tensor(frame, device):
    return torch.from_numpy(frame/255.).float()[None, None].to(device)

def crop_image(img,xC,yC,sizeX,sizeY):
    #y,x,z = img.shape
    startx = xC - sizeX//2
    starty = yC - sizeY//2  
    
    img_out = img[starty:(starty+sizeY),startx:(startx+sizeX),:]
    
    while(img_out.shape[0] == 0 or img_out.shape[1]==0):
        sizeX = sizeX - 20        
        sizeY = sizeY - 20
        startx = xC - sizeX//2
        starty = yC - sizeY//2
        img_out = img[starty:(starty+sizeY),startx:(startx+sizeX),:]
    
    return img_out,sizeX1

def read_image2(image_int, device, resize, rotation, resize_float):
    image = cv2.cvtColor(image_int, cv2.COLOR_BGR2GRAY)
    if image is None:
        return None, None, None
    w, h = image.shape[1], image.shape[0]
    w_new, h_new = process_resize(w, h, resize)
    scales = (float(w) / float(w_new), float(h) / float(h_new))

    if resize_float:
        image = cv2.resize(image.astype('float32'), (w_new, h_new))
    else:
        image = cv2.resize(image, (w_new, h_new)).astype('float32')

    if rotation != 0:
        image = np.rot90(image, k=rotation)
        if rotation % 2:
            scales = scales[::-1]

    inp = frame2tensor(image, device)
    return image, inp, scales

def get_matrice_transformation(image0_frag,image1_fre,rotation,device,resize,resize_float,do_viz,Tx,Ty,sizeX1):
    image0, inp0 , scales0 = read_image2(
        image0_frag , device, resize, 0, resize_float)
    image1, inp1, scales1 = read_image2(
        image1_fre , device, resize, 0, resize_float)
    
    fact_i_0 = image0_frag.shape[0] / image0.shape[0]
    fact_j_0 = image0_frag.shape[1] / image0.shape[1]
    
    fact_i_1 = image1_fre.shape[0] / image1.shape[0]
    fact_j_1 = image1_fre.shape[1] / image1.shape[1]
    
    
    # Perform the matching.
    pred = matching({'image0': inp0, 'image1': inp1})
    pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
    kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
    matches, conf = pred['matches0'], pred['matching_scores0']
    
    # Keep the matching keypoints.
    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    mconf = conf[valid]
    '''
    plt.figure(figsize=(10, 10))
    plt.imshow(image0,cmap='gray')
    plt.plot(mkpts0[:, 0], mkpts0[:, 1], "og", markersize=5)  # og:shorthand for green circle
    plt.axis('off')
    plt.show()
    '''
    mkpts0, mkpts1 = filteringBlackAreaKeypoints(image0,mkpts0,mkpts1)
    
    nbr_matches = len( mkpts0)
    
    
    
    if ( len(mkpts0) < 3  ):
        #print('Error : not enough matches, {} matches'.format(len(mkpts0)))
        return -1,0,0, False

    

    #print(mkpts0.shape)
    #print(len(mkpts0))
        
    
    mkpts0[:, 1] *= fact_i_0
    mkpts0[:, 0] *= fact_j_0
    
    mkpts1[:, 1] *= fact_i_1
    mkpts1[:, 0] *= fact_j_1
    '''
    plt.figure(figsize=(10, 10))
    plt.imshow(image0_frag,cmap='gray')
    plt.plot(mkpts0[:, 0], mkpts0[:, 1], "og", markersize=5)  # og:shorthand for green circle
    plt.axis('off')
    plt.show()   
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image1_fre,cmap='gray')
    plt.plot(mkpts1[:, 0], mkpts1[:, 1], "og", markersize=5)  # og:shorthand for green circle
    plt.axis('off')
    plt.show()
    '''
    
    
    
    startx = Tx - sizeX1//2 
    starty = Ty - sizeY1//2
    
    #print(starty,startx)
    #print(image_original.shape)
    
    mkpts1[:, 0] += startx
    mkpts1[:, 1] += starty
    
    '''
    plt.figure(figsize=(10, 10))
    plt.imshow(image_original[(int(mkpts1[1, 0]-n)):(int(mkpts1[1, 0]+n+1)),(int(mkpts1[1, 1])-n):(int(mkpts1[1, 1])+n+1)])
    plt.axis('off')
    plt.show() 
    '''
    
    
    #print(mkpts0)
    L_y, L_x , L_z  = image0_frag.shape
    copy_mkpts0 = np.copy(mkpts0)
    if (rotation == 90):
        #image0_frag_rot = np.rot90(image0_frag)
        #print(copy_mkpts0)
        mkpts0[:, 0] = L_x - copy_mkpts0[:, 1]
        mkpts0[:, 1] = copy_mkpts0[:, 0] 
    elif (rotation == 180):
        #print(copy_mkpts0)
        #image0_frag_rot = np.rot90(image0_frag)
        #image0_frag_rot = np.rot90(image0_frag_rot)
        mkpts0[:, 0] = L_x - copy_mkpts0[:, 0]
        mkpts0[:, 1] = L_y - copy_mkpts0[:, 1] 
    elif (rotation == 270):
        #image0_frag_rot = np.rot90(image0_frag)
        #image0_frag_rot = np.rot90(image0_frag_rot)
        #image0_frag_rot = np.rot90(image0_frag_rot)
        #print(copy_mkpts0)
        mkpts0[:, 0] = copy_mkpts0[:, 1]
        mkpts0[:, 1] = L_y - copy_mkpts0[:, 0] 
    
    #print(image_original[int(mkpts1[0, 1]),int(mkpts1[0, 0]),:])
    #print(np.where(image_original[(int(mkpts1[1, 0]-5)):(int(mkpts1[1, 0]+6)),(int(mkpts1[1, 1])-5):(int(mkpts1[1, 1])+6)] == a))
    '''
    plt.figure(figsize=(30, 30))
    plt.imshow(image_original)
    plt.plot(mkpts1[:, 0], mkpts1[:, 1], "og", markersize=1)  # og:shorthand for green circle
    plt.axis('off')
    plt.show()  
    '''
    np.random.seed(0)
    
    #model1, inliers = ransac(
    #    (mkpts0, mkpts1),
    #    ProjectiveTransform, min_samples=4,
    #    residual_threshold=4, max_trials=10000
    #)
    
    #print('model1',model1)
    
    #print('inliers',inliers)
    
    
    transformation_matrix, rigid_mask = cv2.estimateAffinePartial2D(mkpts0, mkpts1)
    
    inliers = np.transpose(rigid_mask[:] ==1 )[0]
    
    #print('transformation_rigid_matrix',transformation_rigid_matrix)
    #print(transformation_rigid_matrix[1,2])
    
    #print('rigid_mask',np.transpose(rigid_mask[:] ==1 )[0])
    
    n_inliers = np.sum(inliers)
    #print('Nbr Matches apres filtrage :',len( mkpts0))
    #print('Number of inliers: %d.' % n_inliers)
    
    if ( n_inliers < 3  ):
        #print('Error : not enough matches, {} matches'.format(len(mkpts0)))
        return -1,0,0, False
    
    if do_viz:
        n_inliers = np.sum(inliers)
        print('Number of inliers: %d.' % n_inliers)
        inlier_keypoints_left = [cv2.KeyPoint(point[0], point[1], 1) for point in mkpts0[inliers]]
        inlier_keypoints_right = [cv2.KeyPoint(point[0], point[1], 1) for point in mkpts1[inliers]]
        placeholder_matches = [cv2.DMatch(idx, idx, 5) for idx in range(n_inliers)]


        image_r1 = cv2.drawMatches(image0_frag, inlier_keypoints_left, image_original, inlier_keypoints_right, placeholder_matches, None)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(image_r1)
        plt.axis('off')
        plt.show()

    #print(transformation_matrix)
    '''
    T_x = transformation_matrix[0,2]
    T_y = transformation_matrix[1,2]
    rot = - np.angle(transformation_matrix[0,0]+transformation_matrix[1,0]*1j)*180/np.pi
    
    if rot > 0 :
        rot=rot-360
    

    
    tx = -T_y + L_x*np.sin(rot*np.pi/180)/2 - L_y*np.cos(rot*np.pi/180)/2
    ty = -T_x - L_x*np.cos(rot*np.pi/180)/2 - L_y*np.sin(rot*np.pi/180)/2
    '''
    #print('Transltation : (',T_x,',', T_y,')')
    #print('Rotation : ', rot)
    
    bool_accompli = True
    
    return transformation_matrix, nbr_matches , n_inliers, bool_accompli
    
    

def get_transformation(image0_frag,image1_fre,device,resize,resize_float,do_viz,Tx,Ty,sizeX1):
    
    rotation = 0    
    transformation_matrix_0, nbr_matches_0 , nbr_inliers_0,bool_accompli_0 = get_matrice_transformation(image0_frag,image1_fre,rotation,device,resize,resize_float,do_viz,Tx,Ty,sizeX1)
    #print('Transltation : (',T_x_0,',', T_y_0,')')
    #print('Rotation : ', rot_0)
    
    
    #print('######################## ROTATION : 90 ################')    
    image0_frag = np.rot90(image0_frag)
    rotation = 90
    transformation_matrix_90, nbr_matches_90 , nbr_inliers_90,bool_accompli_90 = get_matrice_transformation(image0_frag,image1_fre,rotation,device,resize,resize_float,do_viz,Tx,Ty,sizeX1)   
    #print('Transltation : (',T_x_90,',', T_y_90,')')
    #print('Rotation : ', rot_90)
    
    
    #print('################ ROTATION : 180 ################')
    image0_frag = np.rot90(image0_frag)
    rotation = 180
    transformation_matrix_180, nbr_matches_180 , nbr_inliers_180,bool_accompli_180 = get_matrice_transformation(image0_frag,image1_fre,rotation,device,resize,resize_float,do_viz,Tx,Ty,sizeX1)    
    #print('Transltation : (',T_x_180,',', T_y_180,')')
    #print('Rotation : ', rot_180)
    
    
    #print('################ ROTATION : 270 ####################')
    image0_frag = np.rot90(image0_frag)
    rotation = 270
    transformation_matrix_270, nbr_matches_270 , nbr_inliers_270,bool_accompli_270 = get_matrice_transformation(image0_frag,image1_fre,rotation,device,resize,resize_float,do_viz,Tx,Ty,sizeX1)    
    #print('Transltation : (',T_x_270,',', T_y_270,')')
    #print('Rotation : ', rot_270)
    
    #rotations = np.array([rot_0,rot_90,rot_180,rot_270])
    #T_x_t = np.array([T_x_0,T_x_90,T_x_180,T_x_270])
    #T_y_t = np.array([T_y_0,T_y_90,T_y_180,T_y_270])
    nbr_matches_t = np.array([nbr_matches_0,nbr_matches_90,nbr_matches_180,nbr_matches_270])
    nbr_inliers_t = np.array([nbr_inliers_0,nbr_inliers_90,nbr_inliers_180,nbr_inliers_270])
    
    #print(rotations)
    #print(T_x_t)
    #print(T_y_t)
    #print(nbr_matches_t)
    #print(nbr_inliers_t)
    
    index_resultat =-1
    
    if(bool_accompli_0 == False and bool_accompli_90 == False and bool_accompli_180 == False and bool_accompli_270 == False ):
        print('Error: Pas possible de trouver Matrice de rotation')
        return 'Nan','Nan','Nan'
    else:
        nbr_inliers_max = np.max(nbr_inliers_t)
        nbr_inliers_max_index = np.where(nbr_inliers_t == nbr_inliers_max)
        #print('len(nbr_inliers_max_index[0])',len(nbr_inliers_max_index[0]))
        
        if(len(nbr_inliers_max_index[0]) > 1 ):
            nbr_matches_t[np.where(nbr_inliers_t != nbr_inliers_max)[0]] = 0
            #print('nbr_matches_t',nbr_matches_t)
            nbr_matches_max = np.max(nbr_matches_t)
            #print('nbr_matches_max',nbr_matches_max)
            nbr_matches_max_index = np.where(nbr_matches_t == nbr_matches_max)
            #print('nbr_matches_max_index',nbr_matches_max_index)            
            index_resultat = nbr_matches_max_index[0][0] 
            #print('nbr_matches_max_index',nbr_matches_max_index)
        else:
            index_resultat = nbr_inliers_max_index[0][0]
            
    #Tx = T_x_t[index_resultat]
    #Ty = T_y_t[index_resultat]
    #rot = rotations[index_resultat]
    if(index_resultat == 0):
        matrice_correct = transformation_matrix_0
    elif(index_resultat == 1):
        matrice_correct = transformation_matrix_90
    elif(index_resultat == 2):
        matrice_correct = transformation_matrix_180
    elif(index_resultat == 3):
        matrice_correct = transformation_matrix_270
        
    print(matrice_correct)
    
    T_x = matrice_correct[0,2]
    T_y = matrice_correct[1,2]
    rot = - np.angle(matrice_correct[0,0]+matrice_correct[1,0]*1j)*180/np.pi
    
    if rot > 0 :
        rot=rot-360
    
    L_y, L_x , L_z  = image0_frag.shape
    
    tx = -T_y + L_x*np.sin(rot*np.pi/180)/2 - L_y*np.cos(rot*np.pi/180)/2
    ty = -T_x - L_x*np.cos(rot*np.pi/180)/2 - L_y*np.sin(rot*np.pi/180)/2
    
    
    return tx, ty, rot

## Choix de la fresque et type de fragmentation

In [4]:
path_fresques = "C:\projects\SuperGluePretrainedNetwork\Fresques"
fresques = [f for f in os.listdir(path_fresques) if os.path.isdir(os.path.join(path_fresques, f))]

count_fresque = 0
################## CHANGER ICI FRESQUE ##################
choix_fresque = 0

#print(os.listdir(path_fresques))

for i in range(len(fresques)):    
    print('Fresque {} : {}'.format(count_fresque,fresques[count_fresque]))
    count_fresque += 1

path_AllFragmentations = os.path.join(path_fresques,fresques[choix_fresque])
AllFragmentations = [f for f in os.listdir(path_AllFragmentations) if os.path.isdir(os.path.join(path_AllFragmentations, f))]

count_fragmentation = 0
################## CHANGER ICI TYPE FRAMENTATION ##################
choix_fragmentation = 0

for i in range(len(AllFragmentations)):    
    print('Fragmentation {} : {}'.format(count_fragmentation,AllFragmentations[count_fragmentation]))
    count_fragmentation += 1
    
path_fragmentation = os.path.join(path_AllFragmentations,AllFragmentations[choix_fragmentation])
print(path_fragmentation)

Fresque 0 : Leonardo-da-Vinci_Ultima-Cena_5193x2926
Fresque 1 : Masolino_GuarigionestorpioeresurrezioneTabita_2112x1080
Fresque 2 : Michelangelo_ThecreationofAdam_1707x775
Fresque 3 : Perugino_Consegnadellechiavi_2347x1438
Fresque 4 : Piero-della-Francesca_ExaltationoftheCross_1239x900
Fragmentation 0 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_13.56.32
Fragmentation 1 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_14.14.54
Fragmentation 2 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_14.35.20
Fragmentation 3 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_14.55.54
Fragmentation 4 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_15.15.49
Fragmentation 5 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_15.36.5
Fragmentation 6 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_15.55.13
Fragmentation 7 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_16.28.9
Fragmentation 8 : Leonardo-da-Vinci_Ultima-Cena_5193x2926_2019-2-28_16.48.9
Fragmentation 9 : Leona

## Obtenir resultats

In [5]:
# Process the file
path_fragments = os.path.join(path_fragmentation,'frag_eroded')
path_list_fragments = os.path.join(path_fragmentation,'fragments.txt')
path_imageFresque = os.path.join(path_AllFragmentations,fresques[choix_fresque]+'.ppm')
path_resultat = os.path.join(path_fragmentation,'resultat.csv')
fichier = open(path_list_fragments, "r")
text = fichier.read()
data = text.split('\n')
image_fresque = np.array(Image.open(path_imageFresque))

# Handle --cache logic.
do_viz = False

resultat = [['Id','Tx','Ty','Rotation']]

for i in range(len(data)-1):
    Tx_hat='Nan'
    Ty_hat='Nan'
    rot_hat='Nan'
    
    id_fragment = int(data[i].split(' ')[0])
    Tx = int(data[i].split(' ')[1])
    Ty = int(data[i].split(' ')[2])
    rot = int(data[i].split(' ')[2])
    
    path = os.path.join(path_fragments,'frag_eroded_{}_color.ppm'.format(id_fragment))
    image_fragment = np.array(Image.open(path))
    sizeX1 = sizeY1 = int(np.mean(image_fragment.shape[0:2]))
    
    image_PartieFresque, sizeX1 = crop_image(image_fresque,Tx,Ty,sizeX1,sizeY1)        
    
    #print('MATRICE TRANSFORMATION DU FRAGMENT {} :'.format(id_fragment))
    
    Tx_hat, Ty_hat, rot_hat = get_transformation(image_fragment,image_PartieFresque,device,resize,resize_float,do_viz,Tx,Ty,sizeX1)   
    
    resultat.append([id_fragment,Tx_hat,Ty_hat,rot_hat])
    
    
    if (Tx_hat != 'Nan' and Ty_hat != 'Nan'and rot_hat != 'Nan'):
        print('Translation : (',Tx_hat,',', Ty_hat,')')
        print('Rotation : ', rot_hat)
        
with open(path_resultat, 'w', newline='') as student_file:
    writer = csv.writer(student_file)
    for i in resultat:        
        writer.writerow(i)


        

[[ 4.18097785e-07  1.00000077e+00  1.41499980e+03]
 [-1.00000077e+00  4.18097785e-07  1.75500010e+03]]
Translation : ( -1546.5001914473594 , -1623.4998830221748 )
Rotation :  -270.00002395521994


KeyboardInterrupt: 